In [5]:
import pandas as pd
import numpy as np
import plotly.express as px
import tensorflow as tf
from utils.preprocessing import get_gray_images
import sys

In [2]:
testing_dir = '../data/C-NMC_Leukemia/testing_data/C-NMC_test_final_phase_data'
training_dir_base = f'../data/C-NMC_Leukemia/training_data/'
fold_0_all_path = 'fold_0/all'
fold_0_hem_path = 'fold_0/hem'

In [8]:
### The code for this lives in notebooks.utils.preprocessing
gray_img_train_cancer = get_gray_images(training_dir_base+fold_0_all_path)
#gray_img_train_healthy = get_gray_images(training_dir_base+fold_0_hem_path)
#gray_img_test = get_gray_images(testing_dir)
len(gray_img_train_cancer)

2397

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

